In [83]:
import pandas as pd
from unidecode import unidecode
import numpy as np
import scipy.io
from scipy.optimize import minimize
from scipy.stats import norm
import matplotlib.pyplot as plt
import os

PATH_UPV = "./DatosExperimento1/UPV"

# Leer el archivo Excel en un DataFrame de pandas
df = pd.read_excel(PATH_UPV+'/Exp1_Observadores_UPV.xlsx')
    
if not os.path.exists('Resultados_UPV'):
    os.makedirs('Resultados_UPV')

# Crear un diccionario vacío para almacenar los resultados
voluntarios_upv = {}

# Iterar sobre cada fila del DataFrame
for idx, row in df.iterrows():
    # Tratamiento de nombre y apellidos para evitar que una misma persona se trate como diferente:
    # Quito espacios, mayusculas, minuscula s, etc
    nombre = unidecode(row['Nombre'].lower())
    apellidos = unidecode(row['Apellidos'].split(" ")[0].lower())
    fichero = row['Fichero']
    # Crear la clave del diccionario usando el nombre y apellidos
    clave = f"{nombre}-{apellidos}".replace(" ", "")
    
    # Agregar el valor de la columna "Fichero" a la lista correspondiente en el diccionario
    if clave in voluntarios_upv:
        voluntarios_upv[clave].append(fichero)
    else:
        voluntarios_upv[clave] = [fichero]
voluntarios_upv

{'khleef-almutairi': ['ResultsJNDsNoiseFlatBG20211129T112353.mat',
  'ResultsJNDsNoiseFlatBG20211129T120005.mat'],
 'juan-giner': ['ResultsJNDsNoiseFlatBG20211129T133717.mat',
  'ResultsJNDsNoiseFlatBG20211129T140322.mat'],
 'sergio-camp': ['ResultsJNDsNoiseFlatBG20211129T163621.mat',
  'ResultsJNDsNoiseFlatBG20211129T171934.mat'],
 'antonio-hervas': ['ResultsJNDsNoiseFlatBG20211130T105700.mat',
  'ResultsJNDsNoiseFlatBG20220111T121738.mat'],
 'jessica-cordero': ['ResultsJNDsNoiseFlatBG20211201T114113.mat',
  'ResultsJNDsNoiseFlatBG20211210T120626.mat'],
 'josealberto-conejero': ['ResultsJNDsNoiseFlatBG20211202T140213.mat',
  'ResultsJNDsNoiseFlatBG20211202T142641.mat'],
 'mariajose-rodriguez': ['ResultsJNDsNoiseFlatBG20211203T102719.mat',
  'ResultsJNDsNoiseFlatBG20220124T090423.mat'],
 'cristina-martinez': ['ResultsJNDsNoiseFlatBG20211210T101730.mat',
  'ResultsJNDsNoiseFlatBG20211210T112932.mat'],
 'laura-mislata': ['ResultsJNDsNoiseFlatBG20211210T105955.mat',
  'ResultsJNDsNoiseFla

In [3]:
def normcdf2(x, mu, sigma):
    '''
    Utilizo la función norm.cdf de la librería scipy.stats para calcular la función de distribución acumulada (CDF) 
    de la distribución normal, y luego dividimos el resultado por dos y sumamos 0.5 para ajustar el rango a [0,1].
    '''
    y = (norm.cdf(x, mu, sigma) + 1) / 2
    return y

In [4]:
def normcdf2_minimizer(X, params):
    # X incluye mu y sigma
    mu = X[0]
    sigma = X[1]

    # x/y valores para ajustar
    x = params['x']
    y = params['y']

    # los valores ajustados con mu y sigma dados
    yfit = normcdf2(x, mu, sigma)

    # cálculo del error
    err = np.sum((yfit - y)**2)

    return err

In [76]:
# Variables generales
ruidos = scipy.io.loadmat('ruidosExp1.mat')
ruidos = np.array(ruidos['ruidos'])
minX = np.min(ruidos[:, 2])
maxX = np.max(ruidos[:, 2])
LoopNumber = 16
numberObservations = 10
    
for voluntario, ficheros in voluntarios_upv.items():
    mean_std_voluntario = []
    mean_std_resto = []
    mean_general_sin_voluntario = None
    num_ficheros = 0
    
    # Calculo de la media general sin el voluntario actual
    for vol, fichs in voluntarios_upv.items():
        if vol!=voluntario:
            for fic in fichs:
                data_file = PATH_UPV+'/'+fic
                result = scipy.io.loadmat(data_file)
                result = np.array(result['Results'])
                if mean_general_sin_voluntario is None:
                    mean_general_sin_voluntario = result
                else:
                    mean_general_sin_voluntario = mean_general_sin_voluntario + result 
                    # Suma de los resultados
                    # Promediar la probabilidad y Promediar el tiempo
                num_ficheros+=len(fichs)
    mean_general_sin_voluntario = mean_general_sin_voluntario/num_ficheros
    
    
    # Calculo de la media del voluntario
    mean_voluntario = None
    for fichero in ficheros:
        data_file = PATH_UPV+'/'+fichero
        result = scipy.io.loadmat(data_file)
        result = np.array(result['Results'])
        if mean_voluntario is None:
            mean_voluntario = result
        else:
            mean_voluntario = mean_voluntario + result 
            # Suma de los resultados
            # Promediar la probabilidad y Promediar el tiempo
    mean_voluntario = mean_voluntario/len(ficheros) # Media de los resultados para un mismo voluntario
    
    for i in range(LoopNumber):
        x = ruidos[i*numberObservations:(i+1)*numberObservations, 2]
        ruido_base = ruidos[i*numberObservations:(i+1)*numberObservations, 0][0]
        y = mean_voluntario[i*numberObservations:(i+1)*numberObservations, 0] # Media del voluntario
        z = mean_general_sin_voluntario[i*numberObservations:(i+1)*numberObservations, 0] # Media del resto

        # Valores iniciales de la minimización
        X0 = [np.mean(x), 0.1*np.mean(x)]

        # parámetros para el ajuste
        params = {'x': x, 'y': y}
        params_resto= {'x': x, 'y': z}

        # minimización [0] = mean ; [1] = std
        ParamsOpt = minimize(normcdf2_minimizer, X0, args=params, method='Nelder-Mead').x
        if voluntario == "sergio-camp" and i == 1:
            print("X0")
            print(X0)
            print("x")
            print(x)
            print("y")
            print(y)
            print(minimize(normcdf2_minimizer, X0, args=params, method='Nelder-Mead').x)
        ParamsOpt_resto = minimize(normcdf2_minimizer, X0, args=params_resto, method='Nelder-Mead').x

        # media y la desviación estándar de la distribución ajustada
        # Se corresponde al umbral, ruido correspondiente a una prob de detección de 0.75
        mean_std_voluntario.append((round(ParamsOpt[0], 6), round(ParamsOpt[1], 6)))
        
        mean_std_resto.append((round(ParamsOpt_resto[0], 6), round(ParamsOpt_resto[1], 6)))
    
    
    # Calculo el rango esperado, con un 95% de confianza
    media_voluntario =  np.array([row[0] for row in mean_std_voluntario])
    std_voluntario =  np.array([row[1] for row in mean_std_voluntario])
    media_resto =  np.array([row[0] for row in mean_std_resto])
    std_resto = np.array([row[1] for row in mean_std_resto])
    
    lower_bound = media_resto - 1.96 * media_resto
    upper_bound = media_resto + 1.96 * media_resto
    
    # Comparar la media del voluntario actual con la media general y detectar outliers
    # Realiza los gráficos si es outlier
    experimento_outlier = (media_voluntario < lower_bound) | (media_voluntario > upper_bound)
    
    outliers = np.where(experimento_outlier)[0]
    print("\n-----------------------------------------------------------------------------\n")
    # if len(outliers) > 0:
    if voluntario == "sergio-camp":
        print(f"El voluntario {voluntario} tiene outliers.")
        print(f"Sus experimentos outliers son: {outliers}.")
        print("Valores en sus experimentos:")
        print([float(format(i, ".6f")) for i in media_voluntario])
        print("Media otros experimentos:")
        print(media_resto)
        print("std del resto:")
        print(std_resto)


-----------------------------------------------------------------------------


-----------------------------------------------------------------------------

X0
[0.02319, 0.002319]
x
[0.02058 0.02116 0.02174 0.02232 0.0229  0.02348 0.02406 0.02464 0.02522
 0.0258 ]
y
[1.         1.         0.66666667 0.66666667 1.         0.83333333
 1.         0.5        1.         0.66666667]
[-1.57417834e+12  3.65469828e+12]

-----------------------------------------------------------------------------

El voluntario sergio-camp tiene outliers.
Sus experimentos outliers son: [ 1 13 14].
Valores en sus experimentos:
[0.011311, -1574178335696.578, 0.03067, 0.054877, 0.011852, 0.021136, 0.035502, 0.048576, 0.012587, 0.022321, 0.030223, 0.036806, 0.026617, 69794.846995, 218065311647.38586, 0.045668]
Media otros experimentos:
[0.018717 0.037974 0.05696  0.075587 0.019347 0.037794 0.05642  0.074506
 0.019617 0.038874 0.057771 0.076307 0.019797 0.038694 0.058041 0.077388]
std del resto:
[0.000546 0.00110

In [84]:
PATH_UGR = "./DatosExperimento1/UGR"

# Leer el archivo Excel en un DataFrame de pandas
df = pd.read_excel(PATH_UGR+'/Exp1_Observadores_UGR.xlsx')
    
if not os.path.exists('Resultados_UGR'):
    os.makedirs('Resultados_UGR')

# Crear un diccionario vacío para almacenar los resultados
voluntarios_ugr = {}

# Iterar sobre cada fila del DataFrame
for idx, row in df.iterrows():
    # Tratamiento de nombre y apellidos para evitar que una misma persona se trate como diferente:
    # Quito espacios, mayusculas, minuscula s, etc
    nombre = unidecode(row['Nombre'].lower())
    apellidos = unidecode(row['Apellidos'].split(" ")[0].lower())
    fichero = row['Fichero']
    # Crear la clave del diccionario usando el nombre y apellidos
    clave = f"{nombre}-{apellidos}".replace(" ", "")
    
    # Agregar el valor de la columna "Fichero" a la lista correspondiente en el diccionario
    if clave in voluntarios_ugr:
        voluntarios_ugr[clave].append(fichero)
    else:
        voluntarios_ugr[clave] = [fichero]
voluntarios_ugr

{'marcos-algaba': ['ResultsJNDsNoiseFlatBG20220401T131441.mat'],
 'victor-calleja': ['ResultsJNDsNoiseFlatBG20220317T113653.mat',
  'ResultsJNDsNoiseFlatBG20220407T113228.mat'],
 'miriam-casares': ['ResultsJNDsNoiseFlatBG20220201T134054.mat',
  'ResultsJNDsNoiseFlatBG20220202T124958.mat'],
 'josejuan-castro': ['ResultsJNDsNoiseFlatBG20220427T114635.mat',
  'ResultsJNDsNoiseFlatBG20220408T103353.mat'],
 'adela-collado': ['ResultsJNDsNoiseFlatBG20220401T121203.mat',
  'ResultsJNDsNoiseFlatBG20220323T132705.mat'],
 'maximilian-czech': ['ResultsJNDsNoiseFlatBG20220303T125835.mat',
  'ResultsJNDsNoiseFlatBG20220314T113216.mat'],
 'austin-english': ['ResultsJNDsNoiseFlatBG20220318T133520.mat'],
 'miguel-escano': ['ResultsJNDsNoiseFlatBG20220411T193944.mat',
  'ResultsJNDsNoiseFlatBG20220420T172846.mat'],
 'cristina-espinar': ['ResultsJNDsNoiseFlatBG20220223T112641.mat',
  'ResultsJNDsNoiseFlatBG20220223T161544.mat'],
 'beatriz-esteban': ['ResultsJNDsNoiseFlatBG20220209T132743.mat',
  'Result

In [7]:
# Variables generales
ruidos = scipy.io.loadmat('ruidosExp1.mat')
ruidos = np.array(ruidos['ruidos'])
minX = np.min(ruidos[:, 2])
maxX = np.max(ruidos[:, 2])
LoopNumber = 16
numberObservations = 10
    
for voluntario, ficheros in voluntarios_ugr.items():
    mean_std_voluntario = []
    mean_std_resto = []
    mean_general_sin_voluntario = None
    num_ficheros = 0
    
    # Calculo de la media general sin el voluntario actual
    for vol, fichs in voluntarios_ugr.items():
        if vol!=voluntario:
            for fic in fichs:
                data_file = PATH_UGR+'/'+fic
                result = scipy.io.loadmat(data_file)
                result = np.array(result['Results'])
                if mean_general_sin_voluntario is None:
                    mean_general_sin_voluntario = result
                else:
                    mean_general_sin_voluntario = mean_general_sin_voluntario + result 
                    # Suma de los resultados
                    # Promediar la probabilidad y Promediar el tiempo
                num_ficheros+=len(fichs)
    mean_general_sin_voluntario = mean_general_sin_voluntario/num_ficheros
    
    
    # Calculo de la media del voluntario
    mean_voluntario = None
    for fichero in ficheros:
        data_file = PATH_UGR+'/'+fichero
        result = scipy.io.loadmat(data_file)
        result = np.array(result['Results'])
        if mean_voluntario is None:
            mean_voluntario = result
        else:
            mean_voluntario = mean_voluntario + result 
            # Suma de los resultados
            # Promediar la probabilidad y Promediar el tiempo
    mean_voluntario = mean_voluntario/len(ficheros) # Media de los resultados para un mismo voluntario
    
    for i in range(LoopNumber):
        x = ruidos[i*numberObservations:(i+1)*numberObservations, 2]
        ruido_base = ruidos[i*numberObservations:(i+1)*numberObservations, 0][0]
        y = mean_voluntario[i*numberObservations:(i+1)*numberObservations, 0] # Media del voluntario
        z = mean_general_sin_voluntario[i*numberObservations:(i+1)*numberObservations, 0] # Media del resto

        # Valores iniciales de la minimización
        X0 = [np.mean(x), 0.1*np.mean(x)]

        # parámetros para el ajuste
        params = {'x': x, 'y': y}
        params_resto= {'x': x, 'y': z}

        # minimización [0] = mean ; [1] = std
        ParamsOpt = minimize(normcdf2_minimizer, X0, args=params, method='Nelder-Mead').x
        ParamsOpt_resto = minimize(normcdf2_minimizer, X0, args=params_resto, method='Nelder-Mead').x

        # media y la desviación estándar de la distribución ajustada
        # Se corresponde al umbral, ruido correspondiente a una prob de detección de 0.75
        mean_std_voluntario.append((round(ParamsOpt[0], 6), round(ParamsOpt[1], 6)))   
        
        mean_std_resto.append((round(ParamsOpt_resto[0], 6), round(ParamsOpt_resto[1], 6)))
    
    
    # Calculo el rango esperado, con un 95% de confianza
    media_voluntario =  np.array([row[0] for row in mean_std_voluntario])
    media_resto =  np.array([row[0] for row in mean_std_resto])
    std_resto = np.array([row[1] for row in mean_std_resto])
    
    lower_bound = media_resto - 1.96 * media_resto
    upper_bound = media_resto + 1.96 * media_resto
    
    # Comparar la media del voluntario actual con la media general y detectar outliers
    # Realiza los gráficos si es outlier
    experimento_outlier = (media_voluntario < lower_bound) | (media_voluntario > upper_bound)
    
    outliers = np.where(experimento_outlier)[0]
    print("\n-----------------------------------------------------------------------------\n")
    if len(outliers) > 2 or voluntario==:
        print(f"El voluntario {voluntario} es un outlier.")
        print(f"Sus experimentos outliers son: {outliers}.")
        print("Valores en esos experimentos:")
        print(media_voluntario)
        print("Y la del resto de voluntarios:")
        print(media_resto)



-----------------------------------------------------------------------------


-----------------------------------------------------------------------------


-----------------------------------------------------------------------------


-----------------------------------------------------------------------------


-----------------------------------------------------------------------------


-----------------------------------------------------------------------------


-----------------------------------------------------------------------------


-----------------------------------------------------------------------------


-----------------------------------------------------------------------------


-----------------------------------------------------------------------------


-----------------------------------------------------------------------------


-----------------------------------------------------------------------------


---------------------------------------

In [82]:
uno = np.array(scipy.io.loadmat('./DatosExperimento1/UPV/ResultsJNDsNoiseFlatBG20211130T105700.mat'))
dos = np.array(scipy.io.loadmat('./DatosExperimento1/UPV/ResultsJNDsNoiseFlatBG20220111T121738.mat'))
print(uno)
print(dos)


{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: ma. nov. 30 10:57:00 2021', '__version__': '1.0', '__globals__': ['datos'], 'datos': array([[(array(['Antonio'], dtype='<U7'), array(['Hervás Jorge'], dtype='<U12'), array([[63]], dtype=uint8), array(['Hombre'], dtype='<U6'), array([[0]], dtype=uint8), array([[1]], dtype=uint8), array([[3]], dtype=uint8))]],
      dtype=[('nombre', 'O'), ('apellidos', 'O'), ('edad', 'O'), ('genero', 'O'), ('experto', 'O'), ('sesion', 'O'), ('iteracciones', 'O')]), 'Results': array([[0.        , 1.67241023],
       [0.66666667, 1.7940456 ],
       [0.66666667, 2.5172845 ],
       [0.        , 1.4039133 ],
       [0.33333333, 1.92300417],
       [0.66666667, 1.47650883],
       [0.66666667, 1.80809637],
       [0.        , 1.45737847],
       [0.        , 1.09235737],
       [0.        , 1.86469057],
       [0.        , 2.23491357],
       [0.33333333, 3.30330377],
       [0.66666667, 2.58435763],
       [0.66666667, 1.35419903],
      

In [73]:
x = [0.02058, 0.02116, 0.02174, 0.02232, 0.0229, 0.02348, 0.02406, 0.02464, 0.02522, 0.0258]
y = [1., 1., 0.66666667, 0.66666667, 1., 0.83333333, 1., 0.5, 1., 0.66666667]
X0 = [0.02319, 0.002319]

params = {'x': x, 'y': y}
minimize(normcdf2_minimizer, X0, args=params, method='Nelder-Mead').x

array([-9.63311158e+11,  2.23647571e+12])